# Roboto SDK - Fetch Topic Data

This notebook shows how to use the Roboto SDK to:
- List log files and their topics in a dataset
- Fetch data for a topic and plot some of it
- Get events created in a dataset
- Fetch slices of topic data from an event and plot it
- Render image topic data that corresponds to the same event

You can use the [Python SDK documentation](https://docs.roboto.ai/reference/python-sdk/roboto/domain/index.html) for more details on any of the functions used below.

In [ ]:
from roboto import Dataset, File, Topic, Event, MessagePath

## List log files and their topics in a dataset
The dataset used below is public and from a collection of drone racing data. You can see the [dataset](https://app.roboto.ai/datasets/ds_w6lve6tl6f16) and [collection](https://app.roboto.ai/collections/cl_pxlseuhim8ym) on Roboto.

In [ ]:
ds = Dataset.from_id("ds_w6lve6tl6f16")
files = ds.list_files()

for file in files:
    print(file.relative_path)
    topics = file.get_topics()
    for topic in topics:
        print(topic.name)

## Fetch data for a topic and plot some of it
In this case, we're getting `/snappy_imu` topic data matching `linear_acceleration` message paths in a log file.

You can also specify `start_time` and `end_time` in `get_data_as_df`. 

In [ ]:
# Get a specific IMU topic from a log file
topic = file.get_topic("/snappy_imu")

# Fetch all of the linear_acceleration data in the IMU topic as a dataframe
df = topic.get_data_as_df(["linear_acceleration"])

df.plot()

## Get events created in a dataset
Events might have been created on the dataset itself, or on underlying files, topics and message paths.

In this case, there's just one topic-level event on `/snappy_imu`.

In [ ]:
events = list(Event.get_by_dataset(ds.dataset_id, transitive=True))

# Show details for one event
event = events[0]
topic = Topic.from_id(event.topic_ids[0])
print(f"{event.event_id} - {event.name} - {topic.name}")
print(f"t0: {event.start_time}")
print(f"tN: {event.end_time}")

## Fetch slices of topic data from an event and plot it
Note, this is a small subset of the data plotted previously. Events make it easy to get specific slices of data.

In [ ]:
# Fetch the linear_acceleration data associated with the topic event
df = event.get_data_as_df(["linear_acceleration"])
df.plot()

## Render image topic data that corresponds to the same event
The event wasn't specifically created on the image topic, but we can still use the event's start and end time to fetch data from other corresponding topics.

In [ ]:
from IPython.display import display, Image
import io
import json

# Get camera topic with image data
camera_topic = file.get_topic("/snappy_cam/stereo_l")

# Fetch image data corresponding to the event start and end time
camera_data = camera_topic.get_data(
    start_time=event.start_time,
    end_time=event.end_time
)

# Show the first 5 images
for i, datum in enumerate(camera_data):
    if i >= 5:
        break
    display(Image(data=datum["data"], width=500))